# Transformación dee los datos y entrenamiento d3el modelo ysando trabajos de entrenamiento gestionados por SageMaker

En este módulo utilizaremos el mismo Dataset y modelo, pero actualizaremos el modelo para utilizar las características de SageMaker para escalar la transformación del Dataset y el entrenamiento del modelo más allá de Jupyter notebook.

Este notebook incluye todos los pasos clave, como el preprocesamiento de datos con SageMaker Processing, y el entrenamiento y despliegue del modelo con  entrenamiento e inferencia alojados en SageMaker. El tuning automático del modelo en SageMaker se utiliza para ajustar los hiperparámetros del modelo. Si utilizamos TensorFlow 2, podemos utilizar el contenedor del framework de TensorFlow 2 preconstruido de Amazon SageMaker con scripts de entrenamiento similares a los que utilizaríamos fuera de SageMaker.

In [12]:
!pip install --upgrade --quiet --disable-pip-version-check sagemaker=='v2.90.0'

In [21]:
import boto3
import os
import sagemaker
import tensorflow as tf

sess = sagemaker.session.Session()
bucket = sess.default_bucket() 
region = boto3.Session().region_name

data_dir = os.path.join(os.getcwd(), 'data')
os.makedirs(data_dir, exist_ok=True)

train_dir = os.path.join(os.getcwd(), 'data/train')
os.makedirs(train_dir, exist_ok=True)

test_dir = os.path.join(os.getcwd(), 'data/test')
os.makedirs(test_dir, exist_ok=True)

raw_dir = os.path.join(os.getcwd(), 'data/raw')
os.makedirs(raw_dir, exist_ok=True)

batch_dir = os.path.join(os.getcwd(), 'data/batch')
os.makedirs(batch_dir, exist_ok=True)

print(f'SageMaker Version: {sagemaker.__version__}')

SageMaker Version: 2.70.0


## SageMaker Processing para transformación del dataset <a class="anchor" id="SageMakerProcessing">

A continuación, importaremos el Dataset y lo transformaremos con SageMaker Processing, que puede utilizarse para procesar terabytes de datos en un clúster gestionado por SageMaker separado de la instancia que ejecuta nuestro Jupyter Kernel Gateway. En un flujo de trabajo típico de SageMaker, los notebooks sólo se utilizan para la creación de prototipos y pueden ejecutarse en instancias relativamente baratas y menos potentes, mientras que las tareas de procesamiento, formación y alojamiento de modelos se ejecutan en instancias separadas y más potentes gestionadas por SageMaker.

SageMaker Processing incluye soporte off-the-shelf para Scikit-learn, así como una opción de Bring Your Own Container (BYOC), por lo que se puede utilizar con muchas tecnologías para transformación de datos y tareas diferentes. Una alternativa a SageMaker Processing es [SageMaker Data Wrangler](https://aws.amazon.com/sagemaker/data-wrangler/), una herramienta visual de preparación de datos integrada en la interfaz de usuario de SageMaker Studio.    

Para trabajar con SageMaker Processing, primero cargaremos el Dataset de Boston Housing, guardaremos las features en bruto y lo subiremos a Amazon S3 para que SageMaker Processing pueda acceder a ellos.

También guardaremos las etiquetas para el entrenamiento y las pruebas.

In [22]:
import numpy as np
from tensorflow.python.keras.datasets import boston_housing
from sklearn.preprocessing import StandardScaler

(x_train, y_train), (x_test, y_test) = boston_housing.load_data()

np.save(os.path.join(raw_dir, 'x_train.npy'), x_train)
np.save(os.path.join(raw_dir, 'x_test.npy'), x_test)
np.save(os.path.join(raw_dir, 'y_train.npy'), y_train)
np.save(os.path.join(raw_dir, 'y_test.npy'), y_test)

s3_prefix = 'tf-2-workflow'
rawdata_s3_prefix = f'{s3_prefix}/data/raw'
raw_s3 = sess.upload_data(path='./data/raw/', key_prefix=rawdata_s3_prefix)

print(raw_s3)

s3://sagemaker-us-east-1-688013747199/tf-2-workflow/data/raw


A continuación, simplemente suministramos un script de preprocesamiento de datos en Python.  Para este ejemplo, estamos usando un contenedor con el framework Scikit-learn pre-construido por SageMaker, que incluye muchas funciones comunes para el procesamiento de datos. Hay pocas limitaciones en cuanto a los tipos de código y operaciones que se pueden ejecutar, y sólo un **contrato mínimo con la API: los datos de entrada y salida deben colocarse en directorios específicos**. Si se hace esto, SageMaker Processing carga automáticamente los datos de entrada desde S3 y carga los datos transformados de vuelta en S3 cuando el trabajo ha concluído.

### NOTE - SageMaker Local Mode

Para ayudarnos a realizar los cambios de código necesarios para adaptar nuestros script a SageMaker, podemos utilizar el [modo local de SageMaker](https://aws.amazon.com/blogs/machine-learning/use-the-amazon-sagemaker-local-mode-to-train-on-your-notebook-instance/) para procesar, entrenar o inferir directamente desde nuestra máquina local.

El modo local del SDK de Python de Amazon SageMaker puede emular entrenamientos de SageMaker en la CPU (una o varias instancias) y en la GPU (una sola instancia) cambiando un solo argumento en los estimadores de TensorFlow, PyTorch o MXNet. Para ello, utiliza Docker compose y NVIDIA Docker. También extraerá los contenedores de Amazon SageMaker TensorFlow, PyTorch o MXNet de Amazon ECR, por lo que necesitarás poder acceder a un repositorio público de Amazon ECR desde tu entorno local.

En este notebook no utilizaremos el modo local de SageMaker, ya que tenemos listos los scripts de procesamiento, entrenamiento y evaluación. Puedes consultar [este repositorio de GitHub](https://github.com/aws-samples/amazon-sagemaker-local-mode) que contiene ejemplos y recursos relacionados que muestran cómo preprocesar, entrenar, depurar tu script de entrenamiento con breakpoint y servir en tu máquina local utilizando el modo local de Amazon SageMaker.

In [23]:
%%writefile preprocessing.py

import glob
import numpy as np
import os
from sklearn.preprocessing import StandardScaler

if __name__=='__main__':
    
    input_files = glob.glob("/opt/ml/processing/input/*.npy")
    print(f'\nINPUT FILE LIST: \n{input_files}\n')
    
    scaler = StandardScaler()
    x_train = np.load(os.path.join('/opt/ml/processing/input', 'x_train.npy'))
    scaler.fit(x_train)
    
    for file in input_files:
        raw = np.load(file)
        # Solo transformamos las columnas de features
        if 'y_' not in file:
            transformed = scaler.transform(raw)
        if 'train' in file:
            if 'y_' in file:
                output_path = os.path.join('/opt/ml/processing/train', 'y_train.npy')
                np.save(output_path, raw)
                print('SAVED LABEL TRAINING DATA FILE\n')
            else:
                output_path = os.path.join('/opt/ml/processing/train', 'x_train.npy')
                np.save(output_path, transformed)
                print('SAVED TRANSFORMED TRAINING DATA FILE\n')
        else:
            if 'y_' in file:
                output_path = os.path.join('/opt/ml/processing/test', 'y_test.npy')
                np.save(output_path, raw)
                print('SAVED LABEL TEST DATA FILE\n')
            else:
                output_path = os.path.join('/opt/ml/processing/test', 'x_test.npy')
                np.save(output_path, transformed)
                print('SAVED TRANSFORMED TEST DATA FILE\n')

Overwriting preprocessing.py


Antes de arrancar el trabajo de procesamiento de SageMaker, instanciamos un objeto `SKLearnProcessor`. Este objeto permite especificar el tipo de instancia a utilizar en el trabajo, así como el número de instancias. Arrancar un cluster es sólo cuestión de establecer `instance_count` a 2 o más, pero nuestra transformación tiene un `StandardScaler` que debe ejecutarse sobre todos los datos de entrenamiento y aplicarse por igual a los datos de entrenamiento y de test. Esto no se puede paralelizar con `scikit-learn`, pero como el Dataset es pequeño, no supone problema.

In [25]:
from sagemaker import get_execution_role
from sagemaker.sklearn.processing import SKLearnProcessor

try:
    execution_role = get_execution_role()
except ValueError:
    execution_role = "AmazonSageMaker-ExecutionRole-20191003T111555"

sklearn_processor1 = SKLearnProcessor(
  framework_version='0.23-1',
  role=execution_role,
  instance_type='ml.t3.medium',
  instance_count=4
)

Ahora estamos listos para ejecutar el trabajo de procesamiento.

Para permitir la distribución de los archivos de datos por igual entre las instancias, podríamosespecificar el tipo de distribución `ShardedByS3Key` en el objeto `ProcessingInput`. Esto aseguraría que si tienes `n` instancias, cada instancia recibirá `1/n` archivos del bucket S3 especificado.

La siguiente celda de código puede tardar unos 10 minutos en ejecutarse, principalmente para configurar el clúster. Podemos revisar el trabajo en la consola Web, en la sección Processing Jobs.

Al final del trabajo, el cluster será desmontado automáticamente por SageMaker.

In [26]:
%%time

from sagemaker.processing import ProcessingInput, ProcessingOutput
from time import gmtime, strftime 

processing_job_name = f'tf-2-workflow-{strftime("%d-%H-%M-%S", gmtime())}'
output_destination = f's3://{bucket}/{s3_prefix}/data'

sklearn_processor1.run(
    code='preprocessing.py',
    job_name=processing_job_name,
    inputs=[ProcessingInput(
        source=raw_s3,
        destination='/opt/ml/processing/input',
        s3_data_distribution_type='ShardedByS3Key'
    )],
    outputs=[
        ProcessingOutput(output_name='train',
            destination=f'{output_destination}/train',
            source='/opt/ml/processing/train'),
        ProcessingOutput(output_name='test',
            destination=f'{output_destination}/test',
            source='/opt/ml/processing/test')
    ]
)

preprocessing_job_description = sklearn_processor1.jobs[-1].describe()


Job Name:  tf-2-workflow-27-14-09-35
Inputs:  [{'InputName': 'input-1', 'AppManaged': False, 'S3Input': {'S3Uri': 's3://sagemaker-us-east-1-688013747199/tf-2-workflow/data/raw', 'LocalPath': '/opt/ml/processing/input', 'S3DataType': 'S3Prefix', 'S3InputMode': 'File', 'S3DataDistributionType': 'ShardedByS3Key', 'S3CompressionType': 'None'}}, {'InputName': 'code', 'AppManaged': False, 'S3Input': {'S3Uri': 's3://sagemaker-us-east-1-688013747199/tf-2-workflow-27-14-09-35/input/code/preprocessing.py', 'LocalPath': '/opt/ml/processing/input/code', 'S3DataType': 'S3Prefix', 'S3InputMode': 'File', 'S3DataDistributionType': 'FullyReplicated', 'S3CompressionType': 'None'}}]
Outputs:  [{'OutputName': 'train', 'AppManaged': False, 'S3Output': {'S3Uri': 's3://sagemaker-us-east-1-688013747199/tf-2-workflow/data/train', 'LocalPath': '/opt/ml/processing/train', 'S3UploadMode': 'EndOfJob'}}, {'OutputName': 'test', 'AppManaged': False, 'S3Output': {'S3Uri': 's3://sagemaker-us-east-1-688013747199/tf-2-w

UnexpectedStatusException: Error for Processing job tf-2-workflow-27-14-09-35: Failed. Reason: AlgorithmError: See job logs for more information

### Dividiendo carga entre varias instancias

En la salida del log del trabajo de Procesamiento de SageMaker de arriba, podemos ver registros diferentes para las dos instancias, ya que cada instancia recibió archivos diferentes. 

Sin el tipo de distribución `ShardedByS3Key`, cada instancia recibiría una copia de **todos** los archivos.  Al repartir los datos de forma equitativa entre `n` instancias, deberíamos percibir un aumento de velocidad de aproximadamente un factor de `n` para la mayoría de las transformaciones de datos stateless.

 Después de guardar los resultados del trabajo localmente, pasaremos al código de entrenamiento e inferencia.

In [ ]:
x_train_in_s3 = f'{output_destination}/train/x_train.npy'
y_train_in_s3 = f'{output_destination}/train/y_train.npy'
x_test_in_s3 = f'{output_destination}/test/x_test.npy'
y_test_in_s3 = f'{output_destination}/test/y_test.npy'

!aws s3 cp {x_train_in_s3} ./data/train/x_train.npy
!aws s3 cp {y_train_in_s3} ./data/train/y_train.npy
!aws s3 cp {x_test_in_s3} ./data/test/x_test.npy
!aws s3 cp {y_test_in_s3} ./data/test/y_test.npy

##  Entrenamiento alojado en SageMaker <a class="anchor" id="SageMakerHostedTraining">

Ahora que hemos preparado Dataset, podemos pasar a la funcionalidad de entrenamiento del modelo de SageMaker.

Con el entrenamiento alojado de SageMaker, el entrenamiento en sí mismo no ocurre en la instancia del notebook tampoco, sino en un clúster separado de máquinas gestionadas por SageMaker. Antes de comenzar el entrenamiento alojado, los datos deben estar en S3, o en un sistema de archivos EFS o FSx para Lustre.

Ahora subiremos a S3, y confirmaremos que la subida fue exitosa.

In [ ]:
s3_prefix = 'tf-2-workflow'

traindata_s3_prefix = f'{s3_prefix}/data/train'
testdata_s3_prefix = f'{s3_prefix}/data/test'

In [ ]:
train_s3 = sess.upload_data(path='./data/train/', key_prefix=traindata_s3_prefix)
test_s3 = sess.upload_data(path='./data/test/', key_prefix=testdata_s3_prefix)

inputs = {'train':train_s3, 'test': test_s3}

print(inputs)

Ahora podemos configurar un objeto `Estimator` para el entrenamiento alojado. Simplemente llamamos a `fit` para iniciar el entrenamiento alojado.

In [ ]:
from sagemaker.tensorflow import TensorFlow

train_instance_type = 'ml.t3.medium'
hyperparameters = {'epochs': 70, 'batch_size': 128, 'learning_rate': 0.01}

hosted_estimator = TensorFlow(
  source_dir='code',
  entry_point='train.py',
  instance_type=train_instance_type,
  instance_count=1,
  hyperparameters=hyperparameters,
  role=sagemaker.get_execution_role(),
  base_job_name='tf-2-workflow',
  framework_version='2.3.1',
  py_version='py37'
)

Después de iniciar el trabajo de entrenamiento alojado con la llamada al método `fit` que aparece a continuación, deberíamos observar queel validation loss converge con cada epoch. ¿Podemos hacerlo mejor? Veremos una forma de hacerlo en el notebook **Ajuste automático del modelo** más adelante.

Mientras tanto, el trabajo de entrenamiento alojado debería tardar unos 3 minutos en completarse.

In [ ]:
hosted_estimator.fit(inputs)

El trabajo de entrenamiento produce un modelo guardado en S3 que podemos obtener. Este es un ejemplo de la modularidad de SageMaker: habiendo entrenado el modelo en SageMaker, ahora puedes sacar el modelo de SageMaker y ejecutarlo en cualquier otro lugar. Alternativamente, podemos desplegar el modelo en un entorno listo para producción utilizando la funcionalidad de endpoints alojados de SageMaker, como se muestra en la sección **Endpoint alojado en SageMaker** a continuación.

Recuperar el modelo de S3 es muy fácil: el estimador de entrenamiento alojado que creó anteriormente almacena una referencia a la ubicación del modelo en S3.  Sólo tienes que copiar el modelo desde S3 utilizando la propiedad `model_data` del estimador y descomprimirlo para inspeccionar el contenido.

In [ ]:
!aws s3 cp {hosted_estimator.model_data} ./model/model.tar.gz

El archivo descomprimido debe incluir los assets requeridos por TensorFlow Serving para cargar el modelo y servirlo, incluyendo un archivo .pb

In [ ]:
!tar -xvzf ./model/model.tar.gz -C ./model

## Automatic Model Tuning <a class="anchor" id="AutomaticModelTuning">

Hasta ahora nos hemos limitado a ejecutar un trabajo de entrenamiento en host sin ningún intento real de ajustar los hiperparámetros para producir un modelo mejor. Seleccionar los valores correctos de los hiperparámetros para entrenar el modelo puede ser difícil, y normalmente lleva mucho tiempo si se hace manualmente. La combinación correcta de hiperparámetros depende de sus datos y algoritmo; algunos algoritmos tienen muchos hiperparámetros diferentes que pueden ser ajustados. Algunos son muy sensibles a los valores de hiperparámetros seleccionados. Y la mayoría tienen una relación no lineal entre el ajuste del modelo y los valores de los hiperparámetros.

SageMaker Automatic Model Tuning ayuda a automatizar el proceso de ajuste de hiperparámetros: ejecuta múltiples trabajos de entrenamiento con diferentes combinaciones de hiperparámetros para encontrar el conjunto con el mejor rendimiento del modelo.

Comenzamos especificando los hiperparámetros que deseamos ajustar, y el rango de valores sobre el que ajustar cada uno. También debemos especificar una métrica objetivo a optimizar: en este caso de uso, nos gustaría minimizar el validation loss.

In [ ]:
from sagemaker.tuner import IntegerParameter, CategoricalParameter, ContinuousParameter, HyperparameterTuner

hyperparameter_ranges = {
  'learning_rate': ContinuousParameter(0.001, 0.2, scaling_type="Logarithmic"),
  'epochs': IntegerParameter(10, 50),
  'batch_size': IntegerParameter(64, 256),
}

metric_definitions = [
  {
    'Name': 'loss',
    'Regex': ' loss: ([0-9\\.]+)'
  },
  {
    'Name': 'val_loss',
    'Regex': ' val_loss: ([0-9\\.]+)'
  }
]

objective_metric_name = 'val_loss'
objective_type = 'Minimize'

A continuación especificamos un objeto `HyperparameterTuner` que toma las definiciones anteriores como parámetros. Cada trabajo de ajuste debe tener un budget o número máximo de trabajos de entrenamiento. Un trabajo de ajuste se completará después de que se hayan ejecutado todos esos trabajos de entrenamiento.

También podemos especificar cuánto paralelismo emplear, en este caso tres trabajos, lo que significa que el trabajo de ajuste se completará después de que se hayan completado dos series de tres trabajos en paralelo.  Para la estrategia de ajuste de Optimización Bayesiana por defecto utilizada aquí, la búsqueda de ajuste se basa en los resultados de grupos anteriores de trabajos de entrenamiento, por lo que no ejecutamos todos los trabajos en paralelo, sino que dividimos los trabajos en grupos de trabajos paralelos.

Hay una contrapartida: si se utilizan más trabajos paralelos se terminará el tuning más pronto, pero probablemente se sacrificará el accuracy.

Ahora podemos lanzar un trabajo de ajuste de hiperparámetros llamando al método `fit` del objeto `HyperparameterTuner`. El trabajo de ajuste puede tardar unos 10 minutos en terminar.  Mientras espera, el estado del job, incluyendo los metadatos y los resultados de los trabajos de entrenamiento individuales dentro del trabajo de ajuste, se pueden revisar en la consola de SageMaker en el panel **Trabajos de ajuste de hiperparámetros**.

In [ ]:
tuner = HyperparameterTuner(
  hosted_estimator,
  objective_metric_name,
  hyperparameter_ranges,
  metric_definitions,
  max_jobs=6,
  max_parallel_jobs=3,
  objective_type=objective_type)

tuning_job_name = f'tf-2-workflow-{strftime("%d-%H-%M-%S", gmtime())}'
tuner.fit(inputs, job_name=tuning_job_name)
tuner.wait()

Una vez finalizado el trabajo de ajuste, podemos utilizar el objeto `HyperparameterTuningJobAnalytics` del SDK SageMaker Python para listar los 5 trabajos de ajuste con mejor rendimiento. Aunque los resultados varían de un trabajo de ajuste a otro, el mejor validation loss del job (columna `FinalObjectiveValue`) probablemente será sustancialmente menor que el validation loss del trabajo de entrenamiento que hicimos anteriormente, sin ningún ajuste aparte de aumentar manualmente el número de epochs.

In [ ]:
tuner_metrics = sagemaker.HyperparameterTuningJobAnalytics(tuning_job_name)
tuner_metrics.dataframe().sort_values(['FinalObjectiveValue'], ascending=True).head(5)

El tiempo total de entrenamiento y el estado de los trabajos pueden comprobarse con las siguientes líneas de código.

Dado que la parada automática anticipada está desactivada por defecto, todos los trabajos de entrenamiento deberían completarse con normalidad.

In [ ]:
total_time = tuner_metrics.dataframe()['TrainingElapsedTimeSeconds'].sum() / 3600
print("The total training time is {:.2f} hours".format(total_time))
tuner_metrics.dataframe()['TrainingJobStatus'].value_counts()

## SageMaker hosted endpoint <a class="anchor" id="SageMakerHostedEndpoint">

Asumiendo que el mejor modelo del trabajo de ajuste es mejor que el modelo producido por el trabajo de entrenamiento del notebook anterior, ahora podríamos desplegar fácilmente ese modelo en producción. 

Una opción conveniente es usar un endpoint alojado en SageMaker, que sirve predicciones en tiempo real del modelo entrenado (Para predicciones asíncronas y offline en grandes Datasets, podríamos usar SageMaker Processing o SageMaker Batch Transform). El endpoint recuperará TensorFlow SavedModel creado durante el entrenamiento y lo desplegará dentro de un contenedor SageMaker TensorFlow Serving. Todo con una línea de código.  

Más específicamente, llamando al método `deploy` del objeto `HyperparameterTuner` que instanciamos arriba, podemos desplegar directamente el mejor modelo del trabajo de ajuste a un endpoint alojado en SageMaker.

In [ ]:
tuning_predictor = tuner.deploy(initial_instance_count=1, instance_type='ml.t3.medium')

Podemos comparar las predicciones generadas por el endpoint con los valores target reales

In [ ]:
results = tuning_predictor.predict(x_test[:10])['predictions'] 
flat_list = [float('%.1f'%(item)) for sublist in results for item in sublist]
print(f'predictions: \t{np.array(flat_list))}')
print(f'target values: \t{y_test[:10].round(decimals=1))}'

Para evitar cargos de facturación por recursos no utilizados, podemos eliminar el endpoint de predicción para liberar las instancias asociadas.

In [ ]:
sess.delete_endpoint(tuning_predictor.endpoint_name)

## Batch Scoring <a class="anchor" id="BatchScoringStep">

El último paso de este proceso es la puntuación por lotes (inferencia/predicción). Los inputs de este paso serán el modelo que hemos entrenado anteriormente y los datos de prueba. Solamente necesitamos un sencillo script de Python para realizar la inferencia por lotes.

In [ ]:
%%writefile batch-score.py

import os
import subprocess
import sys
import numpy as np
import pathlib
import tarfile

def install(package):
    subprocess.check_call([sys.executable, "-m", "pip", "install", package])

if __name__ == "__main__":
    
    install('tensorflow==2.3.1')
    model_path = f"/opt/ml/processing/model/model.tar.gz"
    with tarfile.open(model_path, 'r:gz') as tar:
      tar.extractall('./model')
    import tensorflow as tf
    model = tf.keras.models.load_model('./model/1')
    test_path = "/opt/ml/processing/test/"
    x_test = np.load(os.path.join(test_path, 'x_test.npy'))
    y_test = np.load(os.path.join(test_path, 'y_test.npy'))
    scores = model.evaluate(x_test, y_test, verbose=2)
    print("\nTest MSE :", scores)
    
    output_dir = "/opt/ml/processing/batch"
    pathlib.Path(output_dir).mkdir(parents=True, exist_ok=True)
    evaluation_path = f"{output_dir}/score-report.txt"
    with open(evaluation_path, 'w') as writer:
      writer.write(f"Test MSE : {scores}")

Aquí utilizaremos SageMaker Processing para realizar el scoring por lotes.

In [ ]:
framework_version = "0.23-1"
batch_instance_type = "ml.t3.medium"
batch_instance_count = 1
batch_scorer = SKLearnProcessor(
  framework_version="0.23-1",
  instance_type="ml.c5.xlarge",
  instance_count=1,
  base_job_name="tf-2-workflow-batch",
  role=execution_role
)

In [ ]:
batch_scorer.run(
  inputs=[
    ProcessingInput(
      source=tuner.best_estimator().model_data,
      destination="/opt/ml/processing/model"
    ),
    ProcessingInput(
      source=sklearn_processor1.latest_job.outputs[1].destination,    # [0] es train, [1] es test
      destination="/opt/ml/processing/test"
    )
  ],
  outputs=[
    ProcessingOutput(output_name="batch",
    source="/opt/ml/processing/batch")
  ],
  code="./batch-score.py"
)

In [ ]:
report_path = f"{batch_scorer.latest_job.outputs[0].destination}/score-report.txt"
!aws s3 cp {report_path} ./score-report.txt --quiet && cat score-report.txt